In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("phl_exoplanet_catalog_2019.csv")
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'phl_exoplanet_catalog_2019.csv'

In [3]:
import os
os.getcwd()


'/Users/manishakatariya/Desktop/habitable-planet-hunter/DataCleaning'

In [4]:
os.listdir()


['.DS_Store', 'missing_analysis.ipynb', '.ipynb_checkpoints']

In [5]:
df = pd.read_csv("phl_exoplanet_catalog_2019.csv")
df.head()


,P_NAME,P_STATUS,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_UPDATED,...,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION,S_CONSTELLATION_ABR,S_CONSTELLATION_ENG,P_RADIUS_EST,P_MASS_EST,P_SEMI_MAJOR_AXIS_EST
0,11 Com b,3.0,6165.86330,-476.74200,476.74200,NaN,NaN,NaN,2007,2014-05-14,...,0,Hot,0,0.083813,Coma Berenices,Com,Berenice's Hair,12.082709,6165.86330,1.29
1,11 UMi b,3.0,4684.78480,-794.57001,794.57001,NaN,NaN,NaN,2009,2018-09-06,...,0,Hot,0,0.082414,Ursa Minor,UMi,Little Bear,12.229641,4684.78480,1.53
2,14 And b,3.0,1525.57440,NaN,NaN,NaN,NaN,NaN,2008,2014-05-14,...,0,Hot,0,0.081917,Andromeda,And,Andromeda,12.848516,1525.57440,0.83
3,14 Her b,3.0,1481.07850,-47.67420,47.67420,NaN,NaN,NaN,2002,2018-09-06,...,0,Cold,0,0.145241,Hercules,Her,Hercules,12.865261,1481.07850,2.93
4,16 Cyg B b,3.0,565.73385,-25.42624,25.42624,NaN,NaN,NaN,1996,2018-09-06,...,1,Warm,0,0.368627,Cygnus,Cyg,Swan,13.421749,565.73385,1.66


In [6]:
df.shape


(4048, 112)

In [7]:
missing_counts = df.isnull().sum()
missing_counts


P_NAME                      0
P_STATUS                    0
P_MASS                   2450
P_MASS_ERROR_MIN         2581
P_MASS_ERROR_MAX         2581
                         ... 
S_CONSTELLATION_ABR         0
S_CONSTELLATION_ENG         0
P_RADIUS_EST                0
P_MASS_EST                  0
P_SEMI_MAJOR_AXIS_EST      70
Length: 112, dtype: int64

In [8]:
missing_percent = (missing_counts / len(df)) * 100
missing_percent.sort_values(ascending=False)


P_DETECTION_MASS    100.0
P_GEO_ALBEDO        100.0
S_MAGNETIC_FIELD    100.0
S_DISC              100.0
P_ATMOSPHERE        100.0
                    ...  
S_DEC                 0.0
P_STATUS              0.0
S_ALT_NAMES           0.0
S_DEC_T               0.0
P_NAME                0.0
Length: 112, dtype: float64

In [ ]:
## Missing Data Analysis

This notebook examines the extent of missing values across columns in the
PHL Exoplanet Catalog dataset.

### Key Observations
- Several columns contain a significant proportion of missing values.
- Some physical and orbital parameters are sparsely populated.
- Columns with extremely high missing percentages may require exclusion
  or careful imputation in future preprocessing steps.
